In [250]:
import math
from enum import Enum
from collections import deque
from bisect import insort
from pyvis.network import Network
import networkx as nx

In [251]:
class Action(Enum):
    UP = 0
    RIGHT = 1
    DOWN = 2
    LEFT = 3

    def __str__(self):
        if self == Action.UP:
            return 'UP'

        if self == Action.RIGHT:
            return 'RIGHT'

        if self == Action.DOWN:
            return 'DOWN'

        if self == Action.LEFT:
            return 'LEFT'


N = 3
BLANK = 0
#initial_state = [1, 2, BLANK, 4, 5, 3, 7, 8, 6]
#initial_state = [BLANK, 2, 3 ,4 ,5 ,6,7,1,8]
#intial_state = [1,2,3,4,5,6,BLANK,7,8]
initial_state = [4, 6, 1, 3, 2, BLANK, 5, 7, 8]
# initial_state = [1,2,3,4,5,6,7,BLANK,8]
goal_state = list(map(lambda x: x % (N*N), range(1, N*N+1)))

G = nx.Graph()
G.add_node(1, label='Hola')
G.add_node(2, label='chau')
G.add_edge(1, 2, action='UP')

# nx.draw_networkx(G)

In [252]:
def manhattan_distance(tile_index, tile, n):
    if tile_index == tile - 1 or tile == BLANK:
        return 0

    tile_row = get_tile_row(tile_index, n)
    tile_col = get_tile_col(tile_index, n)

    tile_row_goal_position = get_tile_row(tile - 1, n)
    tile_col_goal_position = get_tile_col(tile - 1, n);

    distance = abs(tile_row - tile_row_goal_position) + abs(tile_col - tile_col_goal_position)
    return distance

def euclidean_distance(tile_index, tile, n):
    if tile_index == tile - 1 or tile == BLANK:
        return 0

    tile_row = get_tile_row(tile_index, n)
    tile_col = get_tile_col(tile_index, n)

    tile_row_goal_position = get_tile_row(tile - 1, n)
    tile_col_goal_position = get_tile_col(tile - 1, n);

    row_distance = abs(tile_row - tile_row_goal_position)
    col_distance = abs(tile_col - tile_col_goal_position)

    distance = math.sqrt(row_distance*row_distance + col_distance*col_distance)
    return distance

def is_tile_in_correct_position(tile_index, tile, n): 
    if tile_index == tile - 1 or tile == BLANK:
        return 0
    return 1

def sum_over_tiles_heuristic(state, calculate, n):
    sum = 0
    for tile_index, tile in enumerate(state):
        sum += calculate(tile_index, tile, n)
    return sum

def manhattan_heuristic(state, n):
    return sum_over_tiles_heuristic(state, manhattan_distance, n)

def euclidean_heuristic(state, n):
    return sum_over_tiles_heuristic(state, euclidean_distance, n)

def misplaced_tiles_heuristic(state, n):
    return sum_over_tiles_heuristic(state, is_tile_in_correct_position, n)

    

In [253]:
class SearchMethod(Enum):
    DFS = 0
    BFS = 1
    DLS = 2
    IDS = 3
    LHS = 4
    GHS = 5
    A_STAR = 6

class Heuristic(Enum):
    MANHATTAN_DISTANCE = manhattan_heuristic
    EUCLIDEAN_DISTANCE = euclidean_heuristic
    MISPLACED_TILES = misplaced_tiles_heuristic

In [254]:
def count_inversions(state, n):
    count = 0

    for i in range(0, n):
        for j in range(i+1, n):
            if state[j] != BLANK and state[i] != BLANK and state[i] > state[j]:
                count += 1

    return count


def is_solvable(initial_state, n):
    inversions = count_inversions(initial_state, n)

    if n % 2 == 1:
        return inversions % 2 == 0
    else:
        blank_index = find_blank_index(initial_state)
        row_from_bottom = N - get_tile_row(blank_index, n)

        if row_from_bottom % 2 == 0:
            return inversions % 2 == 1
        else:
            return inversions % 2 == 0


def get_next_state(state, action, n):
    new_state = state.copy()
    blank_index = find_blank_index(state)

    if action == Action.UP:
        new_state[blank_index], new_state[blank_index -
                                          n] = new_state[blank_index-n], new_state[blank_index]
    elif action == Action.RIGHT:
        new_state[blank_index], new_state[blank_index +
                                          1] = new_state[blank_index+1], new_state[blank_index]
    elif action == Action.DOWN:
        new_state[blank_index], new_state[blank_index +
                                          n] = new_state[blank_index+n], new_state[blank_index]
    elif action == Action.LEFT:
        new_state[blank_index], new_state[blank_index -
                                          1] = new_state[blank_index-1], new_state[blank_index]

    return new_state


def get_actions(state, n):
    blank_index = find_blank_index(state)
    blank_col = blank_index % n
    actions = []

    if blank_col > 0:
        actions.append(Action.LEFT)
    if blank_index <= n*(n - 1) - 1:
        actions.append(Action.DOWN)
    if blank_col < n-1:
        actions.append(Action.RIGHT)
    if blank_index >= n:
        actions.append(Action.UP)

    return actions


def is_goal_state(state):
    return state == goal_state


def is_blank(tile):
    return tile == BLANK


def find_blank_index(state):
    return state.index(BLANK)


def get_tile_col(tile_index, n):
    return tile_index % n


def get_tile_row(tile_index, n):
    return math.floor(tile_index / n)


In [255]:
class Node:
    id = -1

    def __init__(self, state, parent, action, path_cost, depth, estimated_cost=0):
        Node.id = Node.id+1
        self.id = Node.id

        self.state = state
        self.parent = parent
        self.action = action
        self.path_cost = path_cost
        self.estimated_cost = estimated_cost
        self.depth = depth

    def __lt__(self, node):
        return self.estimated_cost < node.estimated_cost


    def __str__(self):
        if self.parent is not None:
            return f"State:{self.state}\tParent State:{self.parent.state}\tAction:{self.action}\tPath Cost:{self.path_cost}\tEstimated Cost:{self.estimated_cost}\n"
        return f"State:{self.state}\tParent State:{self.parent}\tAction:{self.action}\tPath Cost:{self.path_cost}\tEstimated Cost:{self.estimated_cost}\n"


TODO: agregar funciones para estadísticas


In [256]:
def uninformed_search(initial_state, n, search_method):
    root = Node(initial_state, None, None, 0, 0)
    frontier = deque([root])

    if search_method == SearchMethod.BFS:
        # queue
        add_to_frontier = frontier.append
    else:
        # stack
        add_to_frontier = frontier.appendleft

    visited_states = {}
    visited_states[tuple(initial_state)] = True
    visited_node_count = 1

    tree = nx.Graph()
    tree.add_node(root.id, label=str(tuple(root.state)), level=root.depth)

    frontier_len = len(frontier)

    while frontier_len > 0:
        node = frontier.popleft()
        visited_states[tuple(node.state)] = True
        visited_node_count += 1
        frontier_len -= 1

        if is_goal_state(node.state):
            return node, tree
        else:
            actions = get_actions(node.state, n)

            for action in actions:
                new_state = get_next_state(node.state, action, n)
                new_node = Node(new_state, node, action,
                                node.path_cost + 1, node.depth+1)

                tree.add_node(new_node.id, label=str(
                    tuple(new_node.state)), level=node.depth)
                tree.add_edge(new_node.id, new_node.parent.id,
                              label=str(new_node.action))

                if not visited_states.get(tuple(new_state)):
                    add_to_frontier(new_node)
                    frontier_len += 1


In [257]:
def depth_limited_search(initial_state, n, max_depth):
    root = Node(initial_state, None, None, 0, 0)
    frontier = deque([root])

    visited_states = {}
    visited_states[tuple(initial_state)] = 0
    visited_node_count = 1

    tree = nx.Graph()
    tree.add_node(root.id, label=str(tuple(root.state)), level=root.depth)

    frontier_len = len(frontier)

    while frontier_len > 0:
        node = frontier.popleft()
        visited_states[tuple(node.state)] = node.depth
        visited_node_count += 1
        frontier_len -= 1

        if node.depth > max_depth:
            continue
        elif is_goal_state(node.state):
            return node, tree

        else:
            actions = get_actions(node.state, n)

            for action in actions:
                new_state = get_next_state(node.state, action, n)
                new_node = Node(new_state, node, action,
                                node.path_cost + 1, node.depth+1)

                tree.add_node(new_node.id, label=str(
                    tuple(new_node.state)), level=node.depth)
                tree.add_edge(new_node.id, new_node.parent.id,
                              label=str(new_node.action))

                if visited_states.get(tuple(new_state)) is None or visited_states.get(tuple(new_state)) > new_node.depth:
                    frontier.appendleft(new_node)
                    frontier_len += 1


In [258]:
def iterative_depth_search(initial_state, n, initial_limit):
    limit = initial_limit
    result = depth_limited_search(initial_state, n, limit)

    if result is None:
        while result is None:
            limit += 1
            result = depth_limited_search(initial_state, n, limit)
    else:
        lower_limit = 0
        upper_limit = result.depth

        while lower_limit <= upper_limit:
            mid = lower_limit + (upper_limit - lower_limit) // 2
            result = depth_limited_search(initial_state, n, mid)

            if result is None:
                lower_limit = mid + 1
            else:
                upper_limit = mid - 1

    return result


In [259]:
def local_heuristic_search(initial_state, n, heuristic):
    root = Node(initial_state, None, None, 0, 0)
    frontier = deque([root])
    visited_node_count = 1

    visited_states = {}
    visited_states[tuple(initial_state)] = True

    tree = nx.Graph()
    frontier_len = len(frontier)
    tree.add_node(root.id, label=str(tuple(root.state)), level=root.depth)

    while frontier_len > 0:
        node = frontier.popleft()
        visited_states[tuple(node.state)] = True
        visited_node_count += 1
        frontier_len -= 1

        if is_goal_state(node.state):
            return node, tree
        else:
            actions = get_actions(node.state, n)
            successors = []

            for action in actions:
                new_state = get_next_state(node.state, action, n)
                new_node = Node(new_state, node, action,
                                node.path_cost + 1, node.depth+1, heuristic(new_state, n))

                tree.add_node(new_node.id, label="Estimated cost: " + str(new_node.estimated_cost) + "\n" + str(
                    tuple(new_node.state)), level=node.depth)
                tree.add_edge(new_node.id, new_node.parent.id,
                              label=str(new_node.action))

                if not visited_states.get(tuple(new_state)):
                    successors.append(new_node)
                    frontier_len += 1

            successors.sort(key=lambda n: n.estimated_cost, reverse=True)
            frontier.extendleft(successors)


In [260]:
def global_frontier_heuristic_search(initial_state, n, f):
    root = Node(initial_state, None, None, 0, 0)
    frontier = deque([root])

    visited_states = {}
    visited_states[tuple(initial_state)] = True
    visited_node_count = 1

    tree = nx.Graph()
    tree.add_node(root.id, label=str(tuple(root.state)), level=root.depth)

    frontier_len = len(frontier)

    while frontier_len > 0:
        node = frontier.popleft()
        visited_states[tuple(node.state)] = True
        visited_node_count += 1
        frontier_len -= 1

        if is_goal_state(node.state):
            return node, tree
        else:
            actions = get_actions(node.state, n)

            for action in actions:
                new_state = get_next_state(node.state, action, n)
                new_node = Node(new_state, node, action,
                                node.path_cost + 1, node.depth+1)
                new_node.estimated_cost = f(new_node)

                tree.add_node(new_node.id, label="Estimated cost: " + str(new_node.estimated_cost) + "\n" + str(
                    tuple(new_node.state)), level=node.depth)
                tree.add_edge(new_node.id, new_node.parent.id,
                              label=str(new_node.action))

                if not visited_states.get(tuple(new_state)):
                    insort(frontier, new_node)
                    frontier_len += 1


In [261]:
def search(initial_state, search_method, n, heuristic=None, max_depth=-1, initial_limit=-1):
    if not is_solvable(initial_state, n):
        print('problem not solvable')
        return None

    if search_method in [SearchMethod.GHS, SearchMethod.A_STAR]:
        if search_method == SearchMethod.GHS:
            w = 1
        else:
            w = 0.5

        def get_f(heuristic, w, n):
            return lambda node: (1-w)*node.path_cost + w*heuristic(node.state, n)
        return global_frontier_heuristic_search(initial_state, n, get_f(heuristic, w, n))
    elif search_method == SearchMethod.LHS:
        return local_heuristic_search(initial_state, n, heuristic)
    elif search_method == SearchMethod.IDS:
        return iterative_depth_search(initial_state, n, initial_limit)
    elif search_method == SearchMethod.DLS:
        return depth_limited_search(initial_state, n, max_depth)
    elif search_method in [SearchMethod.BFS, SearchMethod.DFS]:
        return uninformed_search(initial_state, n, search_method)


In [276]:
node, tree = search(initial_state, SearchMethod.A_STAR, N, Heuristic.MANHATTAN_DISTANCE)

g=Network(notebook=True, layout='hierarchical')
g.from_nx(tree)

g.show('example.html')